# Getting and processing data

This week, we will cover the topic of getting and processing data. Given a research problem, where can you find the relevant data? How do you obtain the data? And how do you actually process the data? This notebook aims to guide you through the process.

**Important**: this notebook requires you to download and install several items. Please install them before class.

## Where to find data

**Curated**

* Corpora (Brown ([NLTK version](http://www.nltk.org/book/ch02.html)), [OANC](http://www.anc.org/data/oanc/download/), [UMBC WebBase](http://ebiquity.umbc.edu/resource/html/id/351))
* Psycholinguistic data (sometimes known as 'norms' in the Psychology literature)
* DBpedia
* Open data (e.g. [Dutch](https://data.overheid.nl/), [American](https://www.data.gov/))

**The web**

* [USENET](http://www.psych.ualberta.ca/~westburylab/downloads/usenetcorpus.download.html)
* [Internet Archive](https://archive.org/)
* [Project Gutenberg](https://www.gutenberg.org/)
* Wikipedia ([dumps](https://dumps.wikimedia.org/), [export]())
* [Web data commons](http://webdatacommons.org/)

**Do it yourself**

* [BootCat](http://bootcat.sslmit.unibo.it/)
* Experiments
* Annotating
* Crowdsourcing
* ...

## How to get the data

### Downloading directly

Here are three ways to download data from the web, each with their own use cases.

* Browser (loads of data available online)
* Command line: `wget` ([manual](https://www.gnu.org/software/wget/manual/wget.html))
* Python: `requests`, `urllib`

If you see some dataset online, or you just want to download a webpage, there is no better way than to use your browser and either save the page (from the File menu), or to right-click and press "save as..". But for more complex cases, you'll want to automate the process. 

The command line `wget` tool is like a swiss pocket knife for downloading stuff in bulk. For example, if you have a list of URLs in a text file called `list_of_urls.txt`, you can just use `wget -i list_of_urls.txt` to download all the files. You can also use the `wget` module in Python. For more complicated procedures, it's easier to just use the `requests` or `urllib` library.

Here is how we downloaded the Linguist List data for this course:

```python
import os
import urllib.request
import time

base_url = 'http://listserv.linguistlist.org/pipermail/linglite/'
years = [str(year) for year in range(1997,2016)]
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']

for year in years:
    # OS-independent way of creating the path to the folder.
    path = os.path.join('..', 'linguistlist', year)
    # Make the necessary folder.
    os.makedirs(path)
    
    for month in months:
        # Update variables.
        filename = '{}-{}.txt.gz'.format(year, month)
        path_with_file = os.path.join(path, filename)
        url = base_url + filename
        
        # Write the data to disk.
        with urllib.request.urlopen(url) as response:
            # Use the 'wb' flag because the response contents are bytes.
            with open(path_with_file, 'wb') as outfile:
                data = response.read()
                outfile.write(data)
        
        # Be nice to the server.
        time.sleep(2)
```

How did we do this?

* First, we went to the [Linguist List archive website](http://listserv.linguistlist.org/pipermail/linglite/). The archive looks nice, but it's a lot of work to download all of those files by hand!
* Then, we inspected the **source** of the webpage. In Firefox, you can do this by going to `Tools/Developer/Page Source`. In Chrome: `View/Developer/View Source`. Most other browsers offer this functionality as well.
* We saw that the URLs for the monthly archives are very regular. This is good, it means that we can exploit this regularity.
* Then, we decided on a local structure: we want to have one folder for every year, in which all the archives for that year are stored. This structure determined the structure of our program.
* If you don't download files often, search online for a good way to do this. Many programmers would be lost without Google/StackOverflow! The first thing we found was the `urllib` library. But a solution using the `requests` library would also be OK! That would look like this:

```python
import requests

# Get the data:
r = requests.get('http://listserv.linguistlist.org/pipermail/linglite/2016-September.txt.gz')

# Use the 'wb' flag because the response contents are bytes.
with open('September.txt.gz','wb') as f:
	# Write the data:
	f.write(r.content)
```

* It turns out that you can use a context manager (`with`-statement) to treat online sources as files. Cool! That means we can use two context managers (1) to get the file from the internet, and (2) to write the file to disk.
* It's good practice to make your computer wait a little between requests. So we used the `sleep` function from the `time` module to wait 2 seconds after each download.


#### Class discussion
This was a simple example that doesn't require us to do any parsing of the webpage itself. But how would you write a function that takes a URL like [this one](http://listserv.linguistlist.org/pipermail/linguist/2016-September/date.html) and returns all job descriptions? What would be your approach (on a high level)? 

We will revisit this problem in the exercises.

### Using an API

An API (*application programming interface*) provides a way for programs to interact with applications running independently. Those applications could either be running on your own computer, or they could be running somewhere else. We will be working with online APIs, specifically APIs providing the interface to some database. 

General guidelines for using APIs:

1. Try to minimize the number of requests you make. Can you be selective before putting in your requests? 
2. Try to spread your requests so that you don't overload the server.
3. Try to cache your results so that you don't request the same thing twice. (Think about multiple sessions and testing your code.)

In short: developers providing APIs are doing us a favor. Acting nice to them is the least we can do.

#### Bare APIs and wrappers

APIs work like this: you send them a request (possibly with some additional information), and they send you the relevant data back. Sometimes you have to send these requests explicitly in your code, but other times there will be a *wrapper* where people have written code to provide a nice interface for you to use.

**Geopy** is a nice example of a wrapper around several geolocation APIs. Read the documentation [here](https://geopy.readthedocs.io/en/1.10.0/). You can install Geopy using `pip install geopy`. 


When there is no wrapper, you just treat the API as if you are downloading something from the URL. Let's go through some examples. Both of these provide output in JSON format.

**Recipepuppy** is a website where you can search for recipes you can make with a particular set of ingredients. The description of their API is [here](http://www.recipepuppy.com/about/api/). So how do we make this work?

In [1]:
# This library comes pre-installed with Anaconda. We use it to send requests to the web.
import requests

# Get the ingredients
ingredients = input('Please enter the ingredients as a comma-separated list.\n')

# Remove spaces if there are any. (This makes the script more robust.)
ingredients.replace(' ','')

# Prepare the API request URL
base_url = "http://www.recipepuppy.com/api/?i="
api_request = base_url + ingredients

# Get the response
response = requests.get(api_request)

# And print it
print(response.content)

Please enter the ingredients as a comma-separated list.bacon,eggs
b'{"title":"Recipe Puppy","version":0.1,"href":"http:\\/\\/www.recipepuppy.com\\/","results":[{"title":"Grandpa Farrell\'s  Scrambled Eggs","href":"http:\\/\\/www.recipezaar.com\\/Grandpa-Farrells-Scrambled-Eggs-123977","ingredients":"bacon, eggs","thumbnail":"http:\\/\\/img.recipepuppy.com\\/686617.jpg"},{"title":"\\nMicrowave Scrambled Eggs Recipe\\n\\n","href":"http:\\/\\/cookeatshare.com\\/recipes\\/microwave-scrambled-eggs-30382","ingredients":"eggs, bacon","thumbnail":"http:\\/\\/img.recipepuppy.com\\/810118.jpg"},{"title":"Quicker-than-Fast Food Egg Sandwich \\r\\n\\r\\n","href":"http:\\/\\/www.kraftfoods.com\\/kf\\/recipes\\/quicker-than-fast-food-egg-sandwich-69201.aspx","ingredients":"english muffin, eggs, bacon","thumbnail":"http:\\/\\/img.recipepuppy.com\\/601028.jpg"},{"title":"Quicker-than-Fast Food Egg Sandwich \\r\\n\\r\\n","href":"http:\\/\\/www.kraftfoods.com\\/kf\\/recipes\\/quicker-than-fast-food-egg-

We know from last week that JSON objects are just like Python dictionaries, and you can load them using the JSON module. Let's try that!

In [2]:
import json

recipe_data = json.loads(response.content)

TypeError: the JSON object must be str, not 'bytes'

.

.

.

Woops! It turns out that data from the internet is in bytes-format. The JSON library really needs it to be a string.
For this, we need to use the `decode` method to turn the bytes into unicode. If this sounds like magic to you, don't worry: this is something all programmers have struggled with at some point. 

For the next class, please watch the video [Pragmatic Unicode, or: How do I stop the pain?](http://nedbatchelder.com/text/unipain.html). And, if you want to learn more about Unicode, read [The Absolute Minimum Every Software Developer Absolutely, Positively Must Know About Unicode and Character Sets (No Excuses!)](http://www.joelonsoftware.com/articles/Unicode.html).

Now, let's just convert the bytes and continue working with the recipe data.

In [6]:
# Decode bytes into utf-8 (unicode).
decoded_data = response.content.decode('utf-8')

# Load the data.
recipe_data = json.loads(decoded_data)

# Print the keys.
print(recipe_data.keys())

dict_keys(['title', 'version', 'href', 'results'])


It worked! A nice way to inspect JSON response dictionaries is to use the built-in pretty printer from the `pprint` library

In [9]:
# Import the pretty printer:
from pprint import pprint

# Print the recipe data:
pprint(recipe_data)

{'href': 'http://www.recipepuppy.com/',
 'results': [{'href': 'http://www.recipezaar.com/Grandpa-Farrells-Scrambled-Eggs-123977',
              'ingredients': 'bacon, eggs',
              'thumbnail': 'http://img.recipepuppy.com/686617.jpg',
              'title': "Grandpa Farrell's  Scrambled Eggs"},
             {'href': 'http://cookeatshare.com/recipes/microwave-scrambled-eggs-30382',
              'ingredients': 'eggs, bacon',
              'thumbnail': 'http://img.recipepuppy.com/810118.jpg',
              'title': '\nMicrowave Scrambled Eggs Recipe\n\n'},
             {'href': 'http://www.kraftfoods.com/kf/recipes/quicker-than-fast-food-egg-sandwich-69201.aspx',
              'ingredients': 'english muffin, eggs, bacon',
              'thumbnail': 'http://img.recipepuppy.com/601028.jpg',
              'title': 'Quicker-than-Fast Food Egg Sandwich \r\n\r\n'},
             {'href': 'http://www.kraftfoods.com/kf/recipes/quicker-than-fast-food-egg-sandwich-69201.aspx?cm_re=1-_-1-_-Re

**Hackernews** is a website where people can post URLs to interesting stories, submit polls, show the community something, or ask the community a question. The description of their API is [here](https://github.com/HackerNews/API). **Question**: what kind of things could you do with this data?

We will use the Hackernews API in the exercises.

Many APIs require you to authenticate yourself to the server, before they actually return any results. This is a means to prevent abuse (e.g. overloading the server). This usually means you have to register for the service in order to get an *API key*. We won't cover these in class (we don't want to force you to register for anything), but know there are many public APIs out there!

## How to process your data

### Processing the data: HTML


### Processing data: NLP tools

The common idea for all NLP tools is that they try to structure or transform text in some meaningful way. The question of which tool you should use is only secondary to the question what you want to achieve. To give you a sense of the things you can achieve with standard NLP techniques, we will now look at two tools that you can use to analyze text: **SpaCy** and **pyspotlight**. 

#### SpaCy: quickly parsing documents

SpaCy provides a small NLP pipeline: it takes a raw document, tokenizes it, tags all the tokens, and parses each sentence. On top of that, it also recognizes different types of entities: numbers, locations, and persons. The advantage of SpaCy is that it is really fast, and it has a good accuracy. The downside is that, at the moment, it only works for English and German. There are other tools available for different languages, but those are a bit more difficult to set up. (We can help you with this; ask us after class.)

**Installing** 

To install SpaCy, enter the following commands on the command line.

* `conda config --add channels spacy` on the command line
* `conda install spacy`. 
* `python -m spacy.en.download` (if this doesn't work, see [here](http://spacy.io/docs/#getting-started) for updated instructions).

**Using SpaCy**

EXAMPLES AND EXERCISES.

#### pyspotlight: 'interpret' sentences using DBpedia

Pyspotlight provides an easy way to use DBpedia Spotlight, which is a service you can use to find DBpedia entities in a text. DBpedia is --roughly-- a machine-readable version of Wikipedia. In short, this tool enables us to figure out which entities a text is about.

**Installing**

To install pyspotlight, enter the following command on the command line.

* `pip install pyspotlight`

**Using pyspotlight**

EXAMPLES AND EXERCISES.

#### Other tools (not covered in class)

Unfortunately we cannot cover all NLP tools in this course. Below is a short list of tools that might be useful to you in the future. You can either use these tools as standalone programs (and then process their output using Python), or you can choose to use a *wrapper* that allows you to call these tools from inside Python.

* Treetagger is a tool for tokenization and part-of-speech tagging in many languages. [Here](https://github.com/miotto/treetagger-python) is a Python interface for it. 
* Stanford CoreNLP is a suite of NLP tools (constituting a full pipeline). [Here](https://github.com/dasmith/stanford-corenlp-python) is a library to interact with those tools.


## Exercises